# AGiXT Completions Tests and Examples

Simply choose your favorite model of choice from the models list and paste it into the `model` variable on the API calls. You can get a list of models below.

Install OpenAI and requests:

```bash
pip install openai requests python-dotenv
```

**Note, you do not need an OpenAI API Key, the API Key is your `AGIXT_API_KEY` for the server if you defined one in your `.env` file.**

## Global definitions and helpers


In [5]:
import openai
import requests
import time
import os
import re
from dotenv import load_dotenv

load_dotenv()

# Set your system message, max tokens, temperature, and top p here, or use the defaults.
AGENT_NAME = "gpt4free"
AGIXT_SERVER = "http://localhost:7437"
SYSTEM_MESSAGE = "The assistant is acting as a creative writer. All of your text responses are transcribed to audio and sent to the user. Be concise with all responses. After the request is fulfilled, end with </s>."
DEFAULT_MAX_TOKENS = 256
DEFAULT_TEMPERATURE = 0.5
DEFAULT_TOP_P = 0.9

# ------------------- DO NOT EDIT BELOW THIS LINE IN THIS CELL ------------------- #
AGIXT_API_KEY = os.getenv("AGIXT_API_KEY", "none")
DEFAULT_LLM = AGENT_NAME
openai.base_url = f"{AGIXT_SERVER}/v1/"
openai.api_key = AGIXT_API_KEY if AGIXT_API_KEY else AGIXT_SERVER
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"{AGIXT_API_KEY}",
    "ngrok-skip-browser-warning": "true",
}


def display_content(content):
    global AGIXT_SERVER
    global HEADERS
    outputs_url = f"{AGIXT_SERVER}/outputs/"
    os.makedirs("outputs", exist_ok=True)
    try:
        from IPython.display import Audio, display, Image, Video
    except:
        print(content)
        return
    if "http://localhost:8091/outputs/" in content:
        if outputs_url != "http://localhost:8091/outputs/":
            content = content.replace("http://localhost:8091/outputs/", outputs_url)
    if "<audio controls>" in content or " " not in content:
        import base64
        from datetime import datetime

        try:
            audio_response = content.split("data:audio/wav;base64,")[1].split('" type')[
                0
            ]
        except:
            audio_response = content
        file_name = f"outputs/{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.wav"
        with open(file_name, "wb") as fh:
            fh.write(base64.b64decode(audio_response))
        display(Audio(filename=file_name, autoplay=True))
    if outputs_url in content:
        urls = re.findall(f"{re.escape(outputs_url)}[^\"' ]+", content)
        urls = urls[0].split("\n\n")
        for url in urls:
            file_name = url.split("/")[-1]
            url = f"{outputs_url}{file_name}"
            data = requests.get(url, headers=HEADERS).content
            if url.endswith(".jpg") or url.endswith(".png"):
                content = content.replace(url, "")
                display(Image(data=data))
            elif url.endswith(".mp4"):
                content = content.replace(url, "")
                display(Video(data=data, autoplay=True))
            elif url.endswith(".wav"):
                content = content.replace(url, "")
                display(Audio(data=data, autoplay=True))
    print(content)

## Embeddings

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/embeddings)


In [ ]:
# Modify this prompt to generate different outputs
prompt = "Tacos are great."

response = openai.embeddings.create(
    input=prompt,
    model=DEFAULT_LLM,
)
print(response.data[0].embedding)

## Vision Test


In [28]:
response = openai.chat.completions.create(
    model=DEFAULT_LLM,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe each stage of this image."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"https://www.visualwatermark.com/images/add-text-to-photos/add-text-to-image-3.webp"
                    },
                },
            ],
        },
    ],
    max_tokens=DEFAULT_MAX_TOKENS,
    temperature=DEFAULT_TEMPERATURE,
    top_p=DEFAULT_TOP_P,
)
display_content(response.choices[0].message.content)

The image depicts a serene scene of a group of birds flying over a body of water during sunset. The birds are silhouetted against the vibrant hues of the sky, which transitions from a deep blue at the horizon to a warm orange near the horizon. The water below mirrors the sky's colors, creating a harmonious blend of blue and orange.

In the foreground, there's a text overlay that reads "Free as a bird." The text is in a cursive font and is colored in a darker shade, which stands out against the lighter background. The positioning of the text suggests it is a focal point of the image, drawing the viewer's attention to the message it conveys.

The overall composition of the image, with the birds flying above the water and the text in the foreground, creates a sense of freedom and tranquility. The image evokes a feeling of freedom and the beauty of nature, as the birds soar freely in the sky and the sun sets, casting a warm glow over the scene.


## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [6]:
# Modify this prompt to generate different outputs
prompt = "Write a short poem about Pikachu with a picture."


response = openai.chat.completions.create(
    model=DEFAULT_LLM,
    messages=[{"role": "user", "content": prompt}],
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(response.choices[0].message.content)

AttributeError: 'list' object has no attribute 'content'

In [15]:
print(response.choices[0].message[0].content)

I'm glad you're asking for a poem about Pikachu, such a beloved Pokémon character! However, I must clarify that as a text-based AI, I'm unable to provide pictures. But I'll certainly create a short poem for you:

In a world where creatures hold electric power,
Pikachu stands out, a shining tower.
Yellow fur, cheeks aglow,
In the forest, or in the snow.

With a jolt, a spark, a cheerful grin,
Pikachu's journey, adventure within.
Thunderbolt, Quick Attack, Iron Tail too,
A faithful friend, always there for you.

Pikachu, oh Pikachu, so full of life,
Brings joy to all, cuts through strife.
In the Pokémon world, a true gem,
A cherished friend, a light in the den.


## Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/completions/create)


In [30]:
# Modify this prompt to generate different outputs
prompt = "Write a haiku about the future."

completion = openai.completions.create(
    model=DEFAULT_LLM,
    prompt=prompt,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    n=1,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(completion.choices[0].text)

In the future, we'll see,
A world of endless possibilities,
Where dreams come true.


## Text to Speech

Any `wav` file in the `voices` directory can be used as a voice.


In [31]:
from pathlib import Path
import base64
import IPython.display as ipd

prompt = "Write a short poem about vikings with a picture."
os.makedirs("outputs", exist_ok=True)
audio_path = os.path.join(os.getcwd(), "outputs", f"test-speech.wav")
speech_file_path = Path(audio_path)
tts_response = openai.audio.speech.create(
    model="tts-1",
    voice="DukeNukem",
    input=prompt,
    extra_body={"language": "en"},
)
audio_content = base64.b64decode(tts_response.content)
speech_file_path.write_bytes(audio_content)
with open(audio_path, "wb") as audio_file:
    audio_file.write(audio_content)

ipd.Audio(speech_file_path)

## Audio to Text


In [32]:
with open(audio_path, "rb") as audio_file:
    transcription = openai.audio.transcriptions.create(model="base", file=audio_file)

print(transcription.text)

 Write a short poem about Vikings with a picture.
